# Imports

In [618]:
import pandas as pd; import seaborn as sb; import statsmodels.formula.api as smf; import pingouin;
from itertools import combinations; import os; from analysis_helpers import *; 
import scipy; from scipy import stats; import os; import math; import warnings
warnings.filterwarnings('ignore')

# Load and organize data

In [619]:
gaze       = pd.read_csv('../parsed_data/full_gaze.csv')
behavioral = pd.read_csv('../parsed_data/full_behavioral.csv')

for s in behavioral['UniqueID'].unique():
    for r in behavioral['Run'].unique():
        behavioral.loc[(behavioral['Run']==r) 
                 & (behavioral['UniqueID']==s) 
                 & (behavioral['Trial Type']=='Memory'), 'Last Cued'] = behavioral[(behavioral['Run']==r) & (behavioral['UniqueID']==s) & (behavioral['Trial Type']=='Presentation') & (behavioral['Trial']==9)]['Cued Category'].item()


In [620]:
behavioral.loc[behavioral['Trial Type']=='Presentation','Cued Face'] = behavioral[behavioral['Trial Type']=='Presentation']['Cued Composite'].str.split('_',n=1, expand=True)[0]
behavioral.loc[behavioral['Trial Type']=='Presentation','Cued Place'] = behavioral[behavioral['Trial Type']=='Presentation']['Cued Composite'].str.split('_',n=1, expand=True)[1]
behavioral.loc[behavioral['Trial Type']=='Presentation','Uncued Face'] = behavioral[behavioral['Trial Type']=='Presentation']['Uncued Composite'].str.split('_',n=1, expand=True)[0]
behavioral.loc[behavioral['Trial Type']=='Presentation','Uncued Place'] = behavioral[behavioral['Trial Type']=='Presentation']['Uncued Composite'].str.split('_',n=1, expand=True)[1]

In [621]:
behavioral.loc[behavioral['Trial Type']=='Presentation','Cued Face'] = behavioral[behavioral['Trial Type']=='Presentation']['Cued Face']+'.jpg'

In [622]:
behavioral.loc[behavioral['Trial Type']=='Presentation','Uncued Face'] = behavioral[behavioral['Trial Type']=='Presentation']['Uncued Face']+'.jpg'

In [623]:
gaze.head(n=2)

,Unnamed: 0,Unnamed: 0.1,avg,fix,lefteye,raw,righteye,state,time,timestamp,...,xRaw_lefteye,yRaw_lefteye,av_x_coord,av_y_coord,Subject,UniqueID,Group,Experiment,Trial,Run
0,0,43696,"{'x': 1041.042, 'y': 501.128}",True,"{'avg': {'x': 989.74, 'y': 494.5082}, 'pcenter...","{'x': 1029.6586, 'y': 493.4627}","{'avg': {'x': 1092.8706, 'y': 545.1093}, 'pcen...",7,994163077,1.538592e+09,...,989.0439,458.6258,1029.65855,493.4627,10,0,1,/sustain,0.0,3
1,1,43697,"{'x': 1039.5768, 'y': 503.2538}",True,"{'avg': {'x': 989.9557, 'y': 494.9846}, 'pcent...","{'x': 1040.4014, 'y': 537.1355}","{'avg': {'x': 1092.2872, 'y': 546.3817}, 'pcen...",7,994163112,1.538592e+09,...,993.0250,508.4023,1040.40140,537.1355,10,0,1,/sustain,0.0,3


In [624]:
behavioral.head(n=2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Subject,Trial Type,Run,Cued Composite,Uncued Composite,Cued Face,Cued Place,...,Attention Button,Rating History,Stimulus Onset,Stimulus End,Attention Probe,Trial,UniqueID,Group,Experiment,Last Cued
0,0,0,150,10,Presentation,3,00926960627Fa_sunapqwoeotxshkukbh.jpg,00076931230fa_sunaafewhewbxmjiurz.jpg,00926960627Fa.jpg,sunapqwoeotxshkukbh.jpg,...,1.0,NaN,1.538592e+09,1.538592e+09,x,0.0,0,1,/sustain,NaN
1,1,1,151,10,Presentation,3,00725941201fa_sunaefgzwiramypjvtd.jpg,00828940307fa_sunahuynahneraqftyz.jpg,00725941201fa.jpg,sunaefgzwiramypjvtd.jpg,...,3.0,NaN,1.538592e+09,1.538592e+09,o,1.0,0,1,/sustain,NaN


In [625]:
def add_gaze(df):

    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''

    for index,row in df.iterrows():
        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        df['av_x_coord'][index]=df.loc[df[cue+cat] == mem_image]['av_x_coord']
                        df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side'].item()

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'av_x_coord'] = df.loc[mem_mask,'av_x_coord'].fillna(np.nan)

    return(df)

# Gaze Plots

In [626]:
# behavioral[(behavioral['av_x_coord'].isnull()) 
#           &(behavioral['Trial Type'] == 'Presentation') 
#           & (behavioral['Group'] != 2)]

In [627]:
# convert to cm

gaze['av_x_coord'] = (59.8/2048)*(gaze[['xRaw_righteye', 'xRaw_lefteye']].mean(axis=1))
gaze['av_y_coord'] = (33.6/1152)*(gaze[['yRaw_righteye', 'yRaw_lefteye']].mean(axis=1))

In [628]:
# filter anything outside of gaze area

gaze = gaze[(gaze['av_x_coord']   < 59.8) 
                                  & (gaze['av_y_coord'] < 33.6)
                                  & (gaze['av_x_coord'] > 0)
                                  & (gaze['av_y_coord'] > 0)]

In [629]:
# make restricted data

restricted_gaze_list = []
restricted_behavioral_list = []

for sub in gaze['UniqueID'].unique():
    # for each unique subject
    
    print('subject: ' +str(sub))
    
    for run in gaze['Run'].unique():
    # for each unique run
        
        for trial in gaze['Trial'].unique():
        # for each trial
        
            # if there are no rows in df with that subject number, that run number, and that trial number
            # where ax_x_coords are above or below 1cm away from center
            # and av_y_coords are above or below 1cm away from center
            
            d = gaze[(gaze['UniqueID'] == sub) 
                   &(gaze['Run'] == run)
                   &(gaze['Trial']==trial)]
            
            if d[(d['av_x_coord']>=(59.8/2)+1)].shape[0]==0 and d[d['av_x_coord']<=(59.8/2)-1].shape[0]==0 and d[d['av_y_coord']>=(33.6/2)+1].shape[0]==0 and d[d['av_y_coord']<=(33.6/2)-1].shape[0]==0:
                
                #if type(d['av_x_coord'].max())  and type(d['av_y_coord'].max()) == float:
#                 print(d['av_x_coord'].max())
#                 print(d['av_y_coord'].max())
#                 print()
            
                restricted_gaze_list.append(gaze[(gaze['UniqueID'] == sub) 
                               &(gaze['Run'] == run)
                               &(gaze['Trial']==trial)])

                restricted_behavioral_list.append(behavioral[(behavioral['UniqueID']==sub)
                                                            &(behavioral['Run']==run)
                                                            &(behavioral['Trial']==trial)
                                                            &(behavioral['Trial Type']=='Presentation')])

                restricted_behavioral_list.append(behavioral[(behavioral['UniqueID']==sub)
                                            &(behavioral['Run']==run)
                                            &(behavioral['Trial Type']=='Memory')])

subject: 0
subject: 1
subject: 2
subject: 3
subject: 4
subject: 5
subject: 6
subject: 7
subject: 8
subject: 9
subject: 10
subject: 11
subject: 12
subject: 13
subject: 14
subject: 15
subject: 16
subject: 17
subject: 18
subject: 19
subject: 20
subject: 21
subject: 22
subject: 23
subject: 24
subject: 25
subject: 26
subject: 27
subject: 28
subject: 29
subject: 30
subject: 31
subject: 32
subject: 33
subject: 34
subject: 35
subject: 36
subject: 37
subject: 38
subject: 39
subject: 40
subject: 41
subject: 42
subject: 43
subject: 44
subject: 45
subject: 46
subject: 47
subject: 48
subject: 49
subject: 50
subject: 51
subject: 52
subject: 53
subject: 54
subject: 55
subject: 56
subject: 57
subject: 58
subject: 59
subject: 60
subject: 61
subject: 62
subject: 63
subject: 64
subject: 65
subject: 66
subject: 67
subject: 68
subject: 69
subject: 70
subject: 71
subject: 72
subject: 73
subject: 74
subject: 75
subject: 76
subject: 77
subject: 78
subject: 79
subject: 80
subject: 81
subject: 82
subject: 83
su

In [632]:
# restricted_gaze = pd.concat(restricted_gaze_list)
# restricted_behavioral = pd.concat(restricted_behavioral_list)


# # group gaze data by participant, run #, and trial #
# # add average x coordinate from gaze data
# gaze_grp2 = gaze_restricted.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()

# # sort both dataframes
# # gaze_grp2   = gaze_grp2.sort_values(by=['UniqueID','Run','Trial'])
# # behavioral2 = restricted_behavioral.sort_values(by=['UniqueID','Run','Trial'])

# for idx,x in gaze_grp2.iterrows():
#     restricted_behavioral.loc[(restricted_behavioral['UniqueID']==x['UniqueID']) 
#                    & (restricted_behavioral['Run']==x['Run']) 
#                    & (restricted_behavioral['Trial']==x['Trial'])
#                    & (restricted_behavioral['Trial Type']=='Presentation'), 'av_x_coord'] = x['av_x_coord']
    
# behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_cued_side'] #- 59.8/2
# behavioral_added2.loc[behavioral_added2['Attention Level']=='None','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='None']['gaze_towards_mem_image'] 
# behavioral_added2.loc[behavioral_added2['Attention Level']=='Category','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='Category']['gaze_towards_mem_image'] 
# behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_mem_image']/(59.8/2)
    

In [633]:
# group gaze data by participant, run #, and trial #
# add average x coordinate from gaze data
gaze_grp = gaze.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()
gaze_grp2 = gaze_restricted.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()

# sort both dataframes
gaze_grp   = gaze_grp.sort_values(by=['UniqueID','Run','Trial'])
gaze_grp2 = gaze_grp2.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()

behavioral = behavioral.sort_values(by=['UniqueID','Run','Trial'])
restricted_behavioral = restricted_behavioral.sort_values(by=['UniqueID','Run','Trial'])

for idx,x in gaze_grp.iterrows():
    behavioral.loc[(behavioral['UniqueID']==x['UniqueID']) 
                   & (behavioral['Run']==x['Run']) 
                   & (behavioral['Trial']==x['Trial'])
                   & (behavioral['Trial Type']=='Presentation'), 'av_x_coord'] = x['av_x_coord']
    
for idx,x in gaze_grp2.iterrows():
    restricted_behavioral.loc[(restricted_behavioral['UniqueID']==x['UniqueID']) 
                   & (restricted_behavioral['Run']==x['Run']) 
                   & (restricted_behavioral['Trial']==x['Trial'])
                   & (restricted_behavioral['Trial Type']=='Presentation'), 'av_x_coord'] = x['av_x_coord']

In [ ]:
# compute av x gaze's difference from screen center
behavioral['gaze_from_center'] = behavioral['av_x_coord'] - 59.8/2
restricted_behavioral['gaze_from_center'] = restricted_behavioral['av_x_coord'] - 59.8/2

# make the difference value negative for opposite facing arrow
behavioral['gaze_towards_cued_side'] = behavioral['gaze_from_center']
behavioral.loc[(behavioral['Cued Side']=='<'),'gaze_towards_cued_side'] = -behavioral.loc[(behavioral['Cued Side']=='<')]['gaze_towards_cued_side']

# make the difference value negative for opposite facing arrow
restricted_behavioral['gaze_towards_cued_side'] = restricted_behavioral['gaze_from_center']
restricted_behavioral.loc[(restricted_behavioral['Cued Side']=='<'),'gaze_towards_cued_side'] = -restricted_behavioral.loc[(restricted_behavioral['Cued Side']=='<')]['gaze_towards_cued_side']

In [ ]:
def add_gaze(df):

    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''

    for index,row in df.iterrows():
        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        df['gaze_towards_cued_side'][index]=df.loc[df[cue+cat] == mem_image]['gaze_towards_cued_side']
                        df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side'].item()

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'gaze_towards_cued_side'] = df.loc[mem_mask,'gaze_towards_cued_side'].fillna(np.nan)

    return(df)

In [ ]:
behavioral_added = behavioral
for subject in behavioral['UniqueID'].unique():
    print(subject)
    behavioral_added[behavioral_added['UniqueID']==subject] = add_gaze(behavioral[behavioral['UniqueID']==subject])

behavioral_added2 = restricted_behavioral
for subject in restricted_behavioral['UniqueID'].unique():
    print(subject)
    behavioral_added2[behavioral_added2['UniqueID']==subject] = add_gaze(restricted_behavioral[restricted_behavioral['UniqueID']==subject])
        

In [ ]:
behavioral_added['gaze_towards_mem_image'] = behavioral_added['gaze_towards_cued_side']
behavioral_added.loc[behavioral_added['Attention Level']=='None','gaze_towards_mem_image'] = -behavioral_added[behavioral_added['Attention Level']=='None']['gaze_towards_mem_image'] 
behavioral_added.loc[behavioral_added['Attention Level']=='Category','gaze_towards_mem_image'] = -behavioral_added[behavioral_added['Attention Level']=='Category']['gaze_towards_mem_image'] 
behavioral_added['gaze_towards_mem_image'] = behavioral_added['gaze_towards_mem_image']/(59.8/2)

behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_cued_side']
behavioral_added2.loc[behavioral_added2['Attention Level']=='None','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='None']['gaze_towards_mem_image'] 
behavioral_added2.loc[behavioral_added2['Attention Level']=='Category','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='Category']['gaze_towards_mem_image'] 
behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_mem_image']/(59.8/2)

In [ ]:
%matplotlib inline

for exp in ['/variabl','/sustain']:

    for group in [1,2]:
        
        print(exp)
        print(group)
        
        data = behavioral_added[(behavioral_added['Experiment']==exp) 
                                & (behavioral_added['Group']==group)].groupby(['UniqueID','Attention Level','Familiarity Rating','Trial Type'],as_index=False).mean()
        
        sb.violinplot(x='Familiarity Rating', y='gaze_towards_mem_image', hue='Attention Level', 
                      data=data[(data['Attention Level'] != 'Novel') & (data['Trial Type'] == 'Memory') ])
        
        plt.ylim(-1, 1)
        plt.show()
        
        print()

In [ ]:
behavioral_added[behavioral_added['Trial Type']=='Memory'][['gaze_towards_mem_image','Attention Level']]

# Tendency to gaze towards cued side

In [ ]:
behavioral_added[behavioral_added['Trial Type']=='Presentation']['gaze_towards_cued_side']

In [ ]:
behav_grp = behavioral_added.groupby(['UniqueID','Experiment','Group', 'Trial Type'], as_index=False).mean()
for exp in ['/sustain','/variabl']:
    for grp in [1,2]:
        print(exp)
        print(grp)
        dat = behav_grp[(behav_grp['Experiment']==exp) & (behav_grp['Group']==grp) & (behav_grp['Trial Type']=='Presentation')]
        print(scipy.stats.ttest_1samp(dat['gaze_towards_cued_side'],0))
        print(np.mean(dat['gaze_towards_cued_side'])/np.std(dat['gaze_towards_cued_side']))
        print()

# Restrict Gaze (+/- 1cm)

In [ ]:
gaze[(gaze['av_y_coord']>=(33.6/2)+1) | (gaze['av_y_coord']<=(33.6/2)-1)].shape

In [ ]:
59.8/2

In [ ]:
33.6/2

In [ ]:
restricted_gaze = pd.concat(restricted_gaze_list)
restricted_behavioral = pd.concat(restricted_behavioral_list)


# group gaze data by participant, run #, and trial #
# add average x coordinate from gaze data
gaze_grp2 = gaze_restricted.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()

# sort both dataframes
# gaze_grp2   = gaze_grp2.sort_values(by=['UniqueID','Run','Trial'])
# behavioral2 = restricted_behavioral.sort_values(by=['UniqueID','Run','Trial'])

for idx,x in gaze_grp2.iterrows():
    restricted_behavioral.loc[(restricted_behavioral['UniqueID']==x['UniqueID']) 
                   & (restricted_behavioral['Run']==x['Run']) 
                   & (restricted_behavioral['Trial']==x['Trial'])
                   & (restricted_behavioral['Trial Type']=='Presentation'), 'av_x_coord'] = x['av_x_coord']
    
behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_cued_side'] #- 59.8/2
behavioral_added2.loc[behavioral_added2['Attention Level']=='None','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='None']['gaze_towards_mem_image'] 
behavioral_added2.loc[behavioral_added2['Attention Level']=='Category','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='Category']['gaze_towards_mem_image'] 
behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_mem_image']/(59.8/2)
    

In [ ]:
lengths = []
trials = []

for sub in gaze_restricted['UniqueID'].unique():
    lengths.append(len(gaze_restricted[gaze_restricted['UniqueID']==sub]["Run"].unique()))
    trials.append(gaze_restricted[gaze_restricted['UniqueID']==sub].shape[0])

In [ ]:
print(np.mean(lengths))
print(np.mean(trials))


### Plot again

In [ ]:
# behavioral dataframe that matches cases where participants did not look more than 1 cm away
gaze_restricted.groupby(['Experiment','Group','UniqueID','Run','Trial']).mean()

In [ ]:
behavioral

In [ ]:
restricted_behavioral

In [ ]:
# group gaze data by participant, run #, and trial #
# add average x coordinate from gaze data
gaze_grp2 = gaze_restricted.groupby(['UniqueID', 'Run', 'Trial'], as_index=False).mean()

# sort both dataframes
# gaze_grp2   = gaze_grp2.sort_values(by=['UniqueID','Run','Trial'])
# behavioral2 = restricted_behavioral.sort_values(by=['UniqueID','Run','Trial'])

for idx,x in gaze_grp2.iterrows():
    restricted_behavioral.loc[(restricted_behavioral['UniqueID']==x['UniqueID']) 
                   & (restricted_behavioral['Run']==x['Run']) 
                   & (restricted_behavioral['Trial']==x['Trial'])
                   & (restricted_behavioral['Trial Type']=='Presentation'), 'av_x_coord'] = x['av_x_coord']
    
behavioral_added2 = restricted_behavioral

for subject in restricted_behavioral['UniqueID'].unique():
    print(subject)
    behavioral_added2[behavioral_added2['UniqueID']==subject] = add_gaze(restricted_behavioral[restricted_behavioral['UniqueID']==subject])
      
    

In [ ]:
behavioral2

In [ ]:
behavioral_added2 = restricted_behavioral

for subject in restricted_behavioral['UniqueID'].unique():
    print(subject)
    behavioral_added2[behavioral_added2['UniqueID']==subject] = add_gaze(restricted_behavioral[restricted_behavioral['UniqueID']==subject])
      
    

In [ ]:
#behavioral_added2

In [ ]:
#behavioral_added2

In [ ]:
behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_cued_side'] #- 59.8/2
behavioral_added2.loc[behavioral_added2['Attention Level']=='None','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='None']['gaze_towards_mem_image'] 
behavioral_added2.loc[behavioral_added2['Attention Level']=='Category','gaze_towards_mem_image'] = -behavioral_added2[behavioral_added2['Attention Level']=='Category']['gaze_towards_mem_image'] 
#behavioral_added2['gaze_towards_mem_image'] = behavioral_added2['gaze_towards_mem_image']/(59.8/2)

In [ ]:
behavioral_added2[['Familiarity Rating','gaze_towards_mem_image','Familiarity Rating','Memory Image']]

In [ ]:
behavioral_added2['gaze_towards_mem_image'].mean()

In [ ]:
sb.boxplot(x='Familiarity Rating', y='gaze_towards_mem_image', hue='Attention Level', 
                      data=behavioral_added2)

In [ ]:
%matplotlib inline

for exp in ['/variabl','/sustain']:

    for group in [1,2]:
        
        print(exp)
        print(group)
        
        data = behavioral_added2[(behavioral_added2['Experiment']==exp) 
                                & (behavioral_added2['Group']==group)].groupby(['UniqueID','Attention Level','Familiarity Rating','Trial Type'],as_index=False).mean()
        
        sb.violinplot(x='Familiarity Rating', y='gaze_towards_mem_image', hue='Attention Level', 
                      data=data[(data['Attention Level'] != 'Novel') & (data['Trial Type'] == 'Memory') ])
        
        plt.ylim(-1, 1)
        plt.show()
        
        print()

In [ ]:
%matplotlib inline

for exp in ['/variabl','/sustain']:

    #for group in [1,2]:
        
    print(exp)
    print(group)

    data = behavioral_added2[(behavioral_added2['Experiment']==exp)].groupby(['UniqueID','Attention Level','Familiarity Rating','Trial Type'],as_index=False).mean()

    sb.violinplot(x='Familiarity Rating', y='gaze_towards_mem_image', hue='Attention Level', 
                  data=data[(data['Attention Level'] != 'Novel') & (data['Trial Type'] == 'Memory') ])

    plt.ylim(-1, 1)
    plt.show()

    print()

# Plot Again

In [ ]:
import itertools

stat_dict_full = {'/sustain':{},  '/variabl':{}}

# color list 
col = ['r','orange','tan','purple','blue','grey']

# cat list
cats = ['Full','Category','Nov_Cued','Side','None','Nov_Un']

# plot settings
sb.set_style("white")
plt.grid(False)
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.xlabel('Attention Level',    fontsize = 20)
plt.ylabel('Familiarity Rating', fontsize = 20)


# for each experiment, for each test group, group and plot

for experiment in ['/sustain', '/variabl']:

    label = experiment 

    d = behavioral_added2[(behavioral_added2['Experiment']==experiment)]


    # VIOLIN PLOT
    data = d.groupby(['UniqueID','Attention Level', 'Category'], as_index = False).mean()
    print(label + ': Average Familiarity by Attention Level')
    sb_plot = sb.violinplot(x='Attention Level', y='Familiarity Rating', 
                 data = data, hue='Category', split=True,
                 order=['Full','Category','Side','None','Novel'])
    sb_plot.set(ylim=(.2, 9))
    ax1 = sb_plot.axes


    ### SIGNIFICANCE STARS FOR PLOTTING ###
    t_draw = {}

#         for c in data['Attention Level'].unique():
# #             if c in(['Nov_Cued','Nov_Un']) and label=='Experiment_2':

# #                 # if comparing novel images from exp2, eliminate participant 28 (all Place-cued as last cued category)
# #                 first  = list(data[(data['Attention Level']==c) & (data['Category']=='Face') & (data['UniqueID']!=28)]['Familiarity Rating'])
# #                 second = list(data[(data['Attention Level']==c) & (data['Category']=='Place') & (data['UniqueID']!=28)]['Familiarity Rating'])

# #             else:
#             first  = list(data[(data['Attention Level']==c) & (data['Category']=='Face')]['Familiarity Rating'])
#             second = list(data[(data['Attention Level']==c) & (data['Category']=='Place')]['Familiarity Rating'])

#             t = scipy.stats.ttest_rel(first, second)

#             if t[1]<.001:
#                 t_draw[c] = '***'

#             elif t[1]<.01:
#                 t_draw[c] = '**'

#             elif t[1]<.05:
#                 t_draw[c] = '*'

#             elif t[1]<.0551:
#                 t_draw[c] = '+'


#     ### SIGNIFICANCE BETWEEN VIOLINS FOR PLOTTING ###
#     stat_dict = {}
#     k = data.groupby(['UniqueID','Attention Level'],as_index=False).mean()

#     for pair in list(itertools.combinations(cats, r=2)):
#         t = stats.ttest_rel(k[k['Attention Level']==pair[0]]['Familiarity Rating'], 
#                             k[k['Attention Level']==pair[1]]['Familiarity Rating'])
#         stat_dict_full[label][pair] = {'t': t.statistic, 'p': t.pvalue}

#         # dictionary where every key is a pair categories with a difference that has p value less than .0551
#         if t[1]<=.0551:
#             stat_dict[pair] = {'t': t.statistic, 'p': t.pvalue}


#     ### CREATE AND ADD SIGNIFICANCE LINES TO PLOT ###
#     for relationship in  ['pos','neg']:

#         plotted_cats = []
#         to_be_plotted = []
#         line_height = 0

#         # FOR EACH CATEGORY
#         for idx,c in enumerate(cats):

#             x = sig_bars(c, cats, stat_dict, sign=relationship)
#             # get all significance lines from this category to every other category

#             for idx,line in enumerate(x):
#             # for each line from this category to another category (starting with the nearest category)

#                 if (line['categories'] not in plotted_cats) and (line!=np.nan) and (type(line['categories'])!=float):
#                 # if there is a difference in the correct direction (pos/neg) and it has not been plotted yet

#                     line['y'] = line['y'] + line_height
#                     # assign the next available height to this line 
#                     # (give its height a boost based on how many lines have been plotted already)

#                     to_be_plotted.append(line)
#                     # append this line to the list of lines to be plotted

#                     plotted_cats.append(line['categories'])
#                     # add this category pair to the list of category pairs that has been plotted

#                     # CONTINUE THE CASCADE OF LINES
#                     # now, start from the category we have just drawn a line to
#                     # and loop through the rest of the categories from there

#                     # first, give the line a new name so we can loop over new lines, without losing our first one
#                     b = line

#                     # as long as there is difference in the correct direction (pos/neg) 
#                     # between the most recent category and the next one
#                     while b['next']!= 0 :

#                         # grab the first category for the line between them
#                         first_cat = b['categories'][0]

#                         # then get the line STARTING from the category we have just drawn a line to, to the NEXT category it differs from
#                         b = sig_bars(b['next'], cats, stat_dict, sign=relationship)[0]

#                         # if there is a difference in the correct direction (pos/neg) that has not been plotted yet
#                         if (b['categories'] not in plotted_cats) and (b != np.nan) and (type(b['categories'])!=float):

#                             # adjust its height so it is the same height as the line that brought us here
#                             b['y'] = b['y'] + line_height

#                             # add this line to the lines we will plot
#                             to_be_plotted.append(b)

#                             # add this category pair to the list of pairs that's been plotted
#                             plotted_cats.append(b['categories'])

#                             # also add the pair of categories containing the original starting category and this ending category
#                             # (so that doesn't get plotted redundantly)
#                             plotted_cats.append((line['categories'][0], b['categories'][1]))

#                             # AND add the relationship from the original starting category to this starting category
#                             # (so that doesn't get plotted redundantly)
#                             plotted_cats.append((first_cat, b['categories'][1]))

#                     line_height = line_height - .4
#                     # adjust height next lines will be drawn at

#         # Plot the lines
#         for each in to_be_plotted:

#             if relationship == 'pos':
#                 ax1.axhline(each['y'], ls='-', xmin = each['x_min'], xmax = each['x_max'], 
#                     linewidth = each['width'], color = col[cats.index(each['categories'][0])])

#             if relationship == 'neg':
#                 ax1.axhline(each['y']-2, ls='-', xmin = each['x_min'], xmax = each['x_max'], 
#                     linewidth = each['width'], color = col[-cats.index(each['categories'][1])])

#     # Plot the stars
#     for stars in t_draw:
#         ax1.text((cats.index(stars)), 4.5, t_draw[stars], horizontalalignment='center', size='large', color='black')

    # save figure
    # plt.savefig(label+"_violin.pdf")

    # show figure
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
behavioral_added2.columns

### Do Behavioral Again

In [ ]:
# for s in behavioral_added2['UniqueID'].unique():
#     for r in behavioral_added2['Run'].unique():
#         behavioral_added2.loc[(behavioral_added2['Run']==r) 
#                  & (behavioral_added2['UniqueID']==s) 
#                  & (behavioral_added2['Trial Type']=='Memory'), 'Last Cued'] = behavioral_added2[(behavioral_added2['Run']==r) & (behavioral_added2['UniqueID']==s) & (behavioral_added2['Trial Type']=='Presentation') & (behavioral_added2['Trial']==9)]['Cued Category'].item()

# --------

In [ ]:
# make the difference value negative for opposite facing arrow
behavioral['gaze_towards_cued_side'] = behavioral['gaze_from_center']
behavioral.loc[(behavioral['Cued Side']=='<'),'gaze_towards_cued_side'] = -behavioral.loc[(behavioral['Cued Side']=='<')]['gaze_towards_cued_side']

In [ ]:
# ttest relative to zero drift! 
behav_grp = behavioral.groupby(['Experiment','Group', 'UniqueID'], as_index=False).mean()

for exp in ['/sustain','/variabl']:
    for group in [1,2]:
        
        print(exp)
        print(group)
        print(scipy.stats.ttest_1samp(behav_grp[(behav_grp['Experiment']==exp)
                                               &(behav_grp['Group']==group)]['gaze_towards_cued_side'],0))
        
        print(np.mean(behav_grp[(behav_grp['Experiment']==exp)&(behav_grp['Group']==group)]['gaze_towards_cued_side'])/np.std(behav_grp[(behav_grp['Experiment']==exp)
                                               &(behav_grp['Group']==group)]['gaze_towards_cued_side']))
        print()

# Data restricted to gaze between images

In [ ]:
gaze.columns

In [ ]:
gaze_restricted = behavioral[abs(behavioral['gaze_from_center'])<4.2752]
gaze_restricted = gaze_restricted.groupby(['UniqueID','Run','Trial'], as_index=False).mean()
    
#     restricted_behavioral = []

#     for s in gaze_restricted['Subject'].unique():
#         for r in gaze_restricted[gaze_restricted['Subject']==s]['Run'].unique():
#             for t in gaze_restricted[(gaze_restricted['Subject']==s) & (gaze_restricted['Run']==r)]['Trial'].unique():
#                 restricted_behavioral.append(exp1[(exp1['Subject']==s) & (exp1['Run']==int(r)) & (exp1['Trial']==float(t))])
    
#     restricted_concat[label] = pd.concat(restricted_behavioral)

In [ ]:
gaze_restricted

# Attended Category vs Unattended Images

In [ ]:
for l in restricted_concat:
    d = restricted_concat[l]

    Cats = []
    Novs = []
    for s in d['UniqueID'].unique():

        Cats.append(d[(d['Subject']==s) & (d['Attention Level'].isin(['Category','Full']))]['Familiarity Rating'].mean())
        Novs.append(d[(d['Subject']==s) & (d['Attention Level']=='None')]['Familiarity Rating'].mean())

    print(l)
    print(scipy.stats.ttest_rel(Cats, Novs))
    print(cohen_d(Cats, Novs))
    print()

# Attended Side vs Unattended Images

In [ ]:
for l in restricted_concat.keys():
    d = restricted_concat[l]

    Sides = []
    Novs = []
    
    for s in d['Subject'].unique():

        Sides.append(d[(d['Subject']==s) & (d['Attention Level'].isin(['Full','Side']))]['Familiarity Rating'].mean())
        Novs.append(d[(d['Subject']==s) & (d['Attention Level']=='None')]['Familiarity Rating'].mean())


    print(scipy.stats.ttest_rel(Sides, Novs))
    print(cohen_d(Sides, Novs))
    print()

# Plot

In [ ]:
gaze_added_mem_dict = {}
gaze_added_mem = []

for data,label in zip([exp1, exp2],['exp1','exp2']):
    print(label)

    for s in data['Subject'].unique():
        print(s)
        for r in data['Run'].unique():
            gaze_added_mem.append(add_gaze(data[(data['Subject']==s) & (data['Run']==r)]))

    gaze_mem = pd.concat(gaze_added_mem)
    gaze_mem.loc[gaze_mem['Attention Level']=='Novel','av_x_coord']=0

    gaze_mem['gaze_towards'] = gaze_mem['av_x_coord'] - (59.8/2)

    gaze_mem.loc[(gaze_mem['Cued Side']=='<') 
                 & (gaze_mem['Attention Level'].isin(['Full', 'Side'])), 'gaze_towards'] = -gaze_mem[(gaze_mem['Cued Side']=='<') 
                                                                                                          & (gaze_mem['Attention Level'].isin(['Full', 'Side']))]['gaze_towards']

    gaze_mem.loc[(gaze_mem['Cued Side']=='>') 
                 & (gaze_mem['Attention Level'].isin(['None', 'Category'])), 'gaze_towards'] = -gaze_mem[(gaze_mem['Cued Side']=='>') & (gaze_mem['Attention Level'].isin(['None', 'Category']))]['gaze_towards']

    gaze_mem_grp = gaze_mem.groupby(['Subject','Attention Level', 'Familiarity Rating','Trial Type'], as_index=False).mean()

    gaze_added_mem_dict[label] = gaze_mem

In [ ]:
%matplotlib inline
for label in ['exp1','exp2']:
    
    gaze_mem_grp = gaze_added_mem_dict[label].groupby(['Subject','Attention Level', 'Familiarity Rating','Trial Type'], as_index=False).mean()

    # scale by screen size
    gaze_mem_grp['scaled_gaze_towards'] = gaze_mem_grp['gaze_towards']/(59.8/2)

    # plot
    sb.set_style('white')

    f = sb.violinplot(x='Familiarity Rating', y='scaled_gaze_towards', 
                  data=gaze_mem_grp[gaze_mem_grp['Attention Level']!='Novel'], hue='Attention Level')

    plt.grid(False)
    plt.rc('xtick', labelsize=20)
    plt.rc('ytick', labelsize=20)
    plt.xlabel('Familiarity Rating',    fontsize = 20)
    plt.ylabel('Horizontal Gaze Drift', fontsize = 20)
    f.set(ylim=(-1, 1))
    plt.show()


In [ ]:
restricted_behavioral[restricted_behavioral['Group']==2].groupby(['UniqueID','Experiment']).count()

restricted_behavioral['Attention Reaction Time (s)'].unique()

In [ ]:
subbies = []

for subject in data_gr['UniqueID'].unique():
    if data_gr[(data_gr['UniqueID']==subject) & (data_gr['Cue Validity']==1)].shape[0]>0:
        if data_gr[(data_gr['UniqueID']==subject) & (data_gr['Cue Validity']==0)].shape[0]>0:
            subbies.append(subject)

In [ ]:
subbies

In [ ]:
subbies
# data[data['UniqueID'].isin([0, 3, 4, 5, 6, 7, 8, 9])]

In [ ]:
data[data['Attention Reaction Time (s)']>2]

In [ ]:
data = restricted_behavioral

# compare average attention probe reaction times (valid versus invalid)

data_gr = data[data['UniqueID'].isin(subbies)].groupby(['UniqueID','Cue Validity', 'Experiment', 'Group'], as_index=False).mean()

for experiment in ['/sustain', '/variabl']:
    
    for group in [1,2]:
        
        dat = data_gr[(data_gr['Experiment']== experiment) & (data_gr['Group']== group)]
    
        print('exp : '+ experiment + ', group : '+str(group))
        print(scipy.stats.ttest_rel(dat[(dat['Cue Validity']==1)]['Attention Reaction Time (s)'], 

                                    dat[(dat['Cue Validity']==0) ]['Attention Reaction Time (s)']))
        print()
        print(scipy.stats.mannwhitneyu(dat[(dat['Cue Validity']==1)]['Attention Reaction Time (s)'], 

                                    dat[(dat['Cue Validity']==0) ]['Attention Reaction Time (s)']))
        
        sb.barplot(data=dat, x='Cue Validity', y='Attention Reaction Time (s)')
        plt.show()

        print()
        
#         print(cohen_d(list(data_gr[(data_gr['Cue Validity']==1) & (data_gr['Experiment'] == experiment)]['Attention Reaction Time (s)']),
#                       list(data_gr[(data_gr['Cue Validity']==0) & (data_gr['Experiment'] == experiment)]['Attention Reaction Time (s)'])))

        print()